# [LVV-T2213] - Look-up Table Application from MTMount Elevation Changes

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2213]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2213

## Requirements

This notebooks require some extra repositories to be installed locally so it can grab some constants and some look-up tables.  
Here is a list of which repos are required to run this notebook:

* [lsst-ts/ts_cRIOpy]
* [lsst-ts/ts_m1m3support]
* [lsst-ts/ts_config_mttcs]
* [lsst-sitcom/M2_FEA]

[lsst-ts/ts_cRIOpy]: https://github.com/lsst-ts/ts_cRIOpy 
[lsst-ts/ts_m1m3support]: https://github.com/lsst-ts/ts_m1m3support
[lsst-ts/ts_config_mttcs]: https://github.com/lsst-ts/ts_config_mttcs
[lsst-sitcom/M2_FEA]: https://github.com/lsst-sitcom/M2_FEA

Since every user has a different setup, the paths might be slightly different. It is recommended to have all the repositories cloned under `$HOME/notebooks`. You might end up with many repositories and adding an extra folder with the name of the organization they belong might help to find them on GitHub later. For example, this repository would be located in `$HOME/notebooks/lsst-sitcom/notebooks_vandv`.  The paths below consider this directory structure but, of course, you are free to organize your folders as you please.

In order to have the required repositories available, open a terminal and run the following commands:

```
git clone https://github.com/lsst-ts/ts_cRIOpy $HOME/notebooks/lsst-ts/ts_cRIOpy
git clone https://github.com/lsst-ts/ts_m1m3support.git $HOME/notebooks/lsst-ts/ts_m1m3support
git clone https://github.com/lsst-ts/ts_config_mttcs $HOME/notebooks/lsst-ts/ts_config_mttcs
git clone https://github.com/lsst-sitcom/M2_FEA $HOME/notebooks/lsst-sitcom/M2_FEA
```

And add these lines to your `$HOME/notebooks/.user_setup` file: 

```
export LSST_DDS_DOMAIN_ID=0
setup -j notebooks_vandv -r $HOME/notebooks/lsst-sitcom/notebooks_vandv
setup -j ts_cRIOpy -r $HOME/notebooks/lsst-ts/ts_cRIOpy
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [ ]:
test_execution = "LVV-EXXXX"

In [ ]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

## Helper Functions

In [ ]:
async def get_data_from_efd(location, start, end):
    """
    Retrieves data relevant for analysis.
    
    Parameters
    ----------
    location : str
        In which Test Stand are you running this test?
    start : str or datetime
        Start of the time-window.
    end : str or datetime 
        End of the time-window.
    """
    if location == "summit":
        client = EfdClient("summit_efd")
    elif location == "tucson":
        client = EfdClient("tucson_teststand_efd")
    else:
        raise ValueError(
            "Location does not match any valid options {summit|tucson}"
        )
    
    _df_mount_el = await client.select_time_series(
        "lsst.sal.MTMount.elevation", 
        fields="actualPosition", 
        start=start.utc, 
        end=end.utc,
    )
    
    _df_m1m3 = await client.select_time_series(
        "lsst.sal.MTM1M3.forceActuatorData",
        fields="*",
        start=start.utc, 
        end=end.utc,
    )

    # M2 B1 
    _df_m2 = await client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields="*",
        start=start.utc, 
        end=end.utc,
    )
    
    # CamHex Y position
    _df_camhex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=1,
        start=start.utc, 
        end=end.utc,
    )
    
    # M2Hex Y position
    _df_m2hex = await client.select_time_series(
        "lsst.sal.MTHexapod.application",
        fields=[
            "position1"
        ],
        index=2,
        start=start.utc, 
        end=end.utc,
    )
    
    # Rename columns
    _df_mount_el = _df_mount_el.rename(
        columns={"actualPosition": "elevation"})
    _df_m1m3 = _df_m1m3.rename(
        columns={c: f"mtm1m3.forceActuatorData.{c}" for c in _df_m1m3.columns}) 
    _df_m2 = _df_m2.rename(
        columns={c: f"mtm2.axialForce.{c}" for c in _df_m2.columns})
    _df_camhex = _df_camhex.rename(
        columns={c: f"mthexapod_1.application.{c}" for c in _df_camhex.columns})
    _df_m2hex = _df_m2hex.rename(
        columns={c: f"mthexapod_2.application.{c}" for c in _df_m2hex.columns})
    
    # Join dataframes
    _df = _df_mount_el
    
    _df = pd.merge(
        _df, 
        _df_m1m3, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
                
    _df = pd.merge(
        _df, 
        _df_m2, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    _df = pd.merge(
        _df, 
        _df_camhex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )

    _df = pd.merge(
        _df, 
        _df_m2hex, 
        left_index=True, 
        right_index=True, 
        how="outer"
    )
    
    return _df

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [ ]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [ ]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Move mtmount to Zenith

Command the mount to elevation = 90, azimuth = 0, so that we can start m1m3 with LUT in mount telemetry mode).

In [ ]:
await mtcs.point_azel(az=0, el=90)
await mtcs.stop_tracking()

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
# If M1M3 was enabled before, disabled it first and enabled again to start fresh
await mtcs.set_state(salobj.State.DISABLED, components=["mtm1m3"])
await mtcs.set_state(salobj.State.ENABLED, components=["mtm1m3"], overrides={"mtm1m3": "Default"})

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Disable and Enable M2 so we can assure to start fresh
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])
await mtcs.set_state(salobj.State.ENABLED, components=["mtm2"])

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [ ]:
# Disable and Enable CamHex so we can assure to start fresh
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_1"])
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_1"])

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compensation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_2)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# After resetting the Camera Hexapod position, we want to make sure that 
# the compesation and non-compensation values are the same.
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

In [ ]:
# Need to have CamHex LUT use mount telemetry
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

## Gather Data - Without AO

  * command the mount to elevation =86 deg, azimuth = 0
  * wait 39s
  * command the mount to elevation = 82 deg, azimuth = 0.

In [ ]:
# Set this to True when you actually want to run this test
if True:
    t_start = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_start.format = "isot"
    print(f"Gathering data - without AO - Start time: {t_start.utc}")

    await asyncio.sleep(39.)

    # Slew to 86 deg
    await mtcs.point_azel(az=0, el=86)
    await asyncio.sleep(39.)

    # Slew to 82 deg
    await mtcs.point_azel(az=0, el=82)
    await asyncio.sleep(39.)

    t_end = time.Time(utils.current_tai(), format="unix", scale="tai")
    t_end.format = "isot"
    print(f"Gathering data - without AO - End time: {t_end.utc}")
    
    await mtcs.stop_tracking()

## Plot Data

In [ ]:
# t_start = "2022-06-06T16:33:47.387"
# t_start = time.Time(t_start, format="isot", scale="tai")

# t_end = "2022-06-06T16:36:25.569"
# t_end = time.Time(t_end, format="isot", scale="tai")

## Plot Optics vs Time

Plot the following as a function of time during the above process:

- mount elevation
- m1m3 actuator 101 z force
- m2 actuator B1 force
- camera hex y position
- m2 hex y position

In [ ]:
df = await get_data_from_efd(
    exec_info.loc, 
    t_start, 
    t_end)

In [ ]:
df["elevation"].dropna()

In [ ]:
fig, axs = plt.subplots(figsize=(10, 10), nrows=3, sharex=True)

axs[0].plot(df["elevation"].dropna(), "k", label="Mount Elevation")
axs[0].set_ylabel("Mount El\n[deg]")

axs[1].plot(df["mtm2.axialForce.applied0"].dropna(), "C1^-", label="applied")
axs[1].plot(df["mtm2.axialForce.lutGravity0"].dropna(), "C2v-", label="Gravity LUT")
axs[1].set_ylabel("M2 Forces\n[--]")

axs[2].plot(df["mthexapod_1.application.position1"].dropna(), "C3x-", label="CamHex Y")
axs[2].plot(df["mthexapod_2.application.position1"].dropna(), "C4+-", label="M2Hex Y")
axs[2].set_ylabel("Hexapod Position\n[um]")

for ax in axs:
    ax.grid(":", alpha=0.5)
    ax.legend()

fig.suptitle(f"{test_execution} - M1M3/M2/Hexs/Elevation vs Time")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_m2_hexs_el_vs_time.png")
plt.show()

In [ ]:
vandv.m1m3.plot_m1m3_and_elevation(df)

## M1M3 Elevation Forces vs LUT

Check the M1M3 elevation forces match what we expect from the implemented LUT.

In [ ]:
elevation = await mtcs.rem.mtmount.tel_elevation.aget(timeout=10.)
print(elevation, "\n")

The look-up table for X has basically zeroes.  
At least for now.  
So it does not make much sense evaluating it now.

The lut for Y has non-zeroes and zeroes values.  
in order to compare, we can drop the zeroes components.

For z, we are good. We can perform a direct comparison.

In [ ]:
lut_el_xforces = vandv.m1m3.lut_elevation_xforces(elevation.actualPosition)
lut_el_yforces = vandv.m1m3.lut_elevation_yforces(elevation.actualPosition)
lut_el_zforces = vandv.m1m3.lut_elevation_zforces(elevation.actualPosition)

In [ ]:
fel = await mtcs.rem.mtm1m3.evt_appliedElevationForces.aget(timeout=10.)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 3))

ax.plot(fel.xForces, "C0^-", label="Applied")
ax.set_ylabel("Elevation xForces [??]")
ax.grid(":", alpha=0.2)
ax.legend()

fig.suptitle(f"{test_execution} - M1M3 Elevation Forces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_xForces.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 6), nrows=2, sharex=True)

axs[0].plot(fel.yForces, "C0^-", label="Applied")
axs[0].plot(lut_el_yforces[lut_el_yforces != 0], "C1v-", label="LUT")
axs[0].set_ylabel("Elevation yForces [??]")
axs[0].grid(":", alpha=0.2)
axs[0].legend()

axs[1].plot(fel.yForces - lut_el_yforces[lut_el_yforces != 0], label="Applied - LUT forces")
axs[1].set_ylabel("Elefation yForces \n difference [??]")
axs[1].grid(":", alpha=0.2)

fig.suptitle(f"{test_execution} - M1M3 Elevation yForces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_yForces.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 6), nrows=2, sharex=True)

axs[0].plot(fel.zForces, "C0^-", label="Applied")
axs[0].plot(lut_el_zforces, "C1v-", label="LUT")
axs[0].set_ylabel("Elevation zForces [??]")
axs[0].grid(":", alpha=0.2)
axs[0].legend()

axs[1].plot(fel.zForces - lut_el_zforces, label="Applied - LUT forces")
axs[1].set_ylabel("Elefation zForces \n difference [??]")
axs[1].grid(":", alpha=0.2)

fig.suptitle(f"{test_execution} - M1M3 Elevation zForces")
fig.tight_layout(h_pad=0.3)
fig.patch.set_facecolor('white')   

fig.savefig(f"plots/{test_execution}_m1m3_fel_zForces.png")
plt.show()

The following plots are extracted from Bo's Notebooks.  
The originals are in [lsst-ts/ts_notebooks/bxin/ptg2m1m3].

[lsst-ts/ts_notebooks/bxin/ptg2m1m3]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/ptg2m1m3/m1m3_diagnostic.ipynb

In [ ]:
fel = await mtcs.rem.mtm1m3.evt_appliedElevationForces.aget(timeout=10.)
faz = await mtcs.rem.mtm1m3.evt_appliedAzimuthForces.aget(timeout=10.)
fth = await mtcs.rem.mtm1m3.evt_appliedThermalForces.aget(timeout=10.)
fba = await mtcs.rem.mtm1m3.evt_appliedBalanceForces.aget(timeout=10.)
fac = await mtcs.rem.mtm1m3.evt_appliedAccelerationForces.aget(timeout=10.)
fve = await mtcs.rem.mtm1m3.evt_appliedVelocityForces.aget(timeout=10.)
fst = await mtcs.rem.mtm1m3.evt_appliedStaticForces.aget(timeout=10.)
fab = await mtcs.rem.mtm1m3.evt_appliedAberrationForces.aget(timeout=10.)
fof = await mtcs.rem.mtm1m3.evt_appliedOffsetForces.aget(timeout=10.)
fao = await mtcs.rem.mtm1m3.evt_appliedActiveOpticForces.aget(timeout=10.)
fapp = await mtcs.rem.mtm1m3.evt_appliedForces.aget(timeout=10.)

ftel = await mtcs.rem.mtm1m3.tel_forceActuatorData.next(flush=True, timeout=10.)

In [ ]:
def plotForces(fel, fba, fst, fao):
    
    # Get the position of the actuators
    fat = np.array(M1M3FATable.FATABLE)
    xact = np.float64(fat[:, M1M3FATable.FATABLE_XPOSITION])
    yact = np.float64(fat[:, M1M3FATable.FATABLE_YPOSITION])
        
    # Create the plot
    fig, ax = plt.subplots(3,1, figsize=(15,8))
    print(fel.xForces)
    ax[0].plot(fel.xForces, '-o', label='elevation');
    ax[0].plot(fba.xForces, label='FB')
    ax[0].plot(fst.xForces, label='static')
    ax[0].plot(ftel.xForce, '-v', label='measured')
    ax[0].legend()
    ax[0].set_title('XForces')
    
    ax[1].plot(fel.yForces, '-o', label='elevation');
    ax[1].plot(fba.yForces, label='FB')
    ax[1].plot(fst.yForces, label='static')
    ax[1].plot(ftel.yForce, '-v', label='measured')
    ax[1].legend()
    ax[1].set_title('YForces')
    ax[2].plot(fel.zForces, '-o', label='elevation');
    ax[2].plot(fba.zForces, label='FB')
    ax[2].plot(fst.zForces, label='static')
    ax[2].plot(fao.zForces, label='AOS')
    ax[2].plot(ftel.zForce, '-v', label='measured')
    ax[2].set_title('ZForces')
    ax[2].legend()
    
    fig2, ax=plt.subplots( 1,3, figsize = [15,4])
    aa = np.array(fao.zForces)
    img = ax[0].scatter(xact, yact, c=aa)
    ax[0].axis('equal')
    ax[0].set_title('AOS forces')
    fig.colorbar(img, ax=ax[0])

    aa = np.array(fel.zForces)
    img = ax[1].scatter(xact, yact, c=aa)
    ax[1].axis('equal')
    ax[1].set_title('elevation forces')
    fig.colorbar(img, ax=ax[1])
    
    aa = np.array(fst.zForces)
    img = ax[2].scatter(xact, yact, c=aa)
    ax[2].axis('equal')
    ax[2].set_title('static forces')
    fig.colorbar(img, ax=ax[2])

In [ ]:
plotForces(fel, fba, fst, fao)

## M2 Elevation Forces vs LUT

Check the M2 elevation forces match what we expect from the implemented LUT.

In [ ]:
vandv.m2.plot_m2_actuators()

In [ ]:
axialForces = await mtcs.rem.mtm2.tel_axialForce.aget(timeout=2)
tangentForces = await mtcs.rem.mtm2.tel_tangentForce.aget(timeout=2)

In [ ]:
vandv.m2.plotM2Forces(axialForces, tangentForces)

## CamHex Vs LUT

Check the camera hexapod LUT compensations match what we expect from the implemented LUT

In [ ]:
a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=1)
elev = a.elevation

await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_1)    
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)

## M2Hex vs LUT

In [ ]:
a = mtcs.rem.mthexapod_1.evt_compensationOffset.get()
elevCoeff, tCoeff = vandv.hexapod.coeffs_from_lut(index=2)
elev = a.elevation

await vandv.hexapod.print_hexapod_uncompensation_values(mtcs.rem.mthexapod_2)
await vandv.hexapod.print_predicted_compensation(elevCoeff, elev)
await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_2)    
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_2)

### Close up

In [ ]:
# Put the telescope back to the original position
# Specially if running at TTS
target = mtcs.radec_from_azel(az=0, el=80)

await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)
await mtcs.stop_tracking()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
if exec_info.loc == "summit":
    await mtcs.standby()

else:
    # Bring the system back to the original state
    await mtcs.set_state(
        state=salobj.State.ENABLED,
        components=[
            "mtm1m3", 
            "mtm2",
            "mthexapod_1",
            "mthexapod_2",
            "mtaos",
        ],
        overrides={
            "mtm1m3": "Default"
        }
    )

## Wrap Up

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3", "mtm2", "mthexapod_1", "mthexapod_2"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm2", "mthexapod_1", "mthexapod_2"])